In [33]:
from tableauhyperapi import HyperProcess, Connection, Telemetry, TableDefinition, QualifiedName

source_file = 'WorldIndicators.hyper'
target_file_pattern = 'WorldIndicators_{0}.hyper'
table_name = QualifiedName('Extract','Extract')
split_expression = 'EXTRACT(YEAR FROM "Year")'

# Start a new Hyper instance
with HyperProcess(Telemetry.DO_NOT_SEND_USAGE_DATA_TO_TABLEAU, 'hypersplitter') as hyper:
    # Connect to the extract we want to split into multiple Extracts
    with Connection(hyper.endpoint) as connection:
        connection.catalog.attach_database(source_file, alias="source")
        namespace_name = connection.catalog.get_table_definition(table_name).name.prefix.simple_name
        split_domain = connection.execute_result_list(f"SELECT DISTINCT {split_expression} FROM source.{table_name}")
        for domain_row in split_domain:
            domain_value = domain_row[0]
            target_file = target_file_pattern.format(domain_value)
            print(f"Creating '{target_file}'...'")
            connection.catalog.create_database(target_file)
            connection.catalog.attach_database(target_file, alias="target")
            connection.execute_command(f"CREATE SCHEMA target.{namespace_name}")
            connection.execute_command(f"""
                CREATE TABLE target.{table_name} AS
                SELECT * FROM source.{table_name}
                WHERE {split_expression} = {domain_value}
                """)
            connection.catalog.detach_database("target")

Creating 'WorldIndicators_2012.hyper'...'
Creating 'WorldIndicators_2007.hyper'...'
Creating 'WorldIndicators_2002.hyper'...'
Creating 'WorldIndicators_2001.hyper'...'
Creating 'WorldIndicators_2010.hyper'...'
Creating 'WorldIndicators_2000.hyper'...'
Creating 'WorldIndicators_2009.hyper'...'
Creating 'WorldIndicators_2005.hyper'...'
Creating 'WorldIndicators_2006.hyper'...'
Creating 'WorldIndicators_2008.hyper'...'
Creating 'WorldIndicators_2004.hyper'...'
Creating 'WorldIndicators_2011.hyper'...'
Creating 'WorldIndicators_2003.hyper'...'
